<a href="https://colab.research.google.com/github/mushhub/my-first-blockchain/blob/main/HL_delta0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import hmac
import hashlib
import time
import base64
from typing import Dict, List, Any

class HyperliquidClient:
    """Hyperliquid API Client for creating delta neutral positions"""

    def __init__(self, private_key: str, base_url: str = "https://api.hyperliquid.xyz"):
        self.private_key = private_key
        self.base_url = base_url

    def get_signature(self, timestamp: int, data: Dict[str, Any]) -> str:
        """Generate signature for authentication"""
        message = f"{timestamp}{json.dumps(data)}"
        signature = hmac.new(
            bytes.fromhex(self.private_key),
            message.encode('utf-8'),
            hashlib.sha256
        ).hexdigest()
        return signature

    def get_auth_header(self, data: Dict[str, Any]) -> Dict[str, str]:
        """Generate authentication header"""
        timestamp = int(time.time() * 1000)
        signature = self.get_signature(timestamp, data)

        return {
            "X-HL-Signature": signature,
            "X-HL-Timestamp": str(timestamp),
            "X-HL-Api-Key": self.private_key[:32]
        }

    def get_account_info(self) -> Dict[str, Any]:
        """Get account information"""
        url = f"{self.base_url}/exchange/v1/user/account"
        response = requests.get(url, headers=self.get_auth_header({}))
        return response.json()

    def get_market_data(self) -> Dict[str, Any]:
        """Get market data for BTC and other assets"""
        url = f"{self.base_url}/exchange/v1/market/meta"
        response = requests.get(url)
        return response.json()

    def place_order(self, coin: str, is_buy: bool, size: float, limit_px: float) -> Dict[str, Any]:
        """Place a limit order"""
        order_data = {
            "coin": coin,
            "is_buy": is_buy,
            "sz": size,
            "limit_px": limit_px,
            "order_type": {
                "limit": {
                    "tif": "Gtc"  # Good 'til canceled
                }
            },
            "reduce_only": False
        }

        payload = {
            "action": {
                "order": order_data
            }
        }

        url = f"{self.base_url}/exchange/v1/orders"
        headers = self.get_auth_header(payload)
        headers["Content-Type"] = "application/json"

        response = requests.post(url, headers=headers, json=payload)
        return response.json()

    def create_delta_neutral_position(self, btc_spot_price: float, position_size_usd: float, leverage: float = 1.0):
        """
        Create a delta neutral position for BTC

        Args:
            btc_spot_price: Current BTC price
            position_size_usd: Position size in USD
            leverage: Leverage to use (default 1.0)
        """
        # Calculate position sizes
        btc_perp_size = position_size_usd / btc_spot_price * leverage

        # Long spot / Short futures position
        print(f"Creating delta neutral position for BTC:")
        print(f"BTC Current Price: ${btc_spot_price}")
        print(f"Position Size: ${position_size_usd}")
        print(f"Leverage: {leverage}x")

        # Place orders
        # 1. Buy BTC-USD-PERP (Long position)
        long_response = self.place_order(
            coin="BTC",
            is_buy=True,
            size=btc_perp_size,
            limit_px=btc_spot_price * 1.002  # Slightly higher price to ensure execution
        )

        # 2. Sell BTC-USD (Short position)
        short_response = self.place_order(
            coin="BTC",
            is_buy=False,
            size=btc_perp_size,
            limit_px=btc_spot_price * 0.998  # Slightly lower price to ensure execution
        )

        print("Orders placed:")
        print(f"Long Position: {long_response}")
        print(f"Short Position: {short_response}")

        return {
            "long_order": long_response,
            "short_order": short_response
        }


def main():
    # Replace with your actual private key
    private_key = "YOUR_PRIVATE_KEY_HERE"

    # Initialize client
    client = HyperliquidClient(private_key)

    # Get current BTC price from an external source or Hyperliquid API
    # This is just a placeholder - you would need to get the actual price
    btc_current_price = 65000.0  # Example price

    # Create delta neutral position with $10,000 and 2x leverage
    position_size_usd = 10000.0
    leverage = 2.0

    result = client.create_delta_neutral_position(
        btc_spot_price=btc_current_price,
        position_size_usd=position_size_usd,
        leverage=leverage
    )

    print("\nDelta neutral position created successfully!")
    print(f"Position is balanced with equal long and short exposure to BTC")


if __name__ == "__main__":
    main()